<a href="https://colab.research.google.com/github/Ashmit45/ash-demo/blob/main/BP_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =================== STEP 1: Install Libraries ===================
!pip install pennylane -q
!pip install tensorflow -q
!pip install scikit-learn -q
!pip install scipy -q

In [ ]:
# =================== STEP 2: Import Libraries ===================
import os, zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers, callbacks
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.fft import fft
from scipy.signal import stft
import pennylane as qml
from pennylane.qnn import KerasLayer
from google.colab import files

In [ ]:
# =================== STEP 3: Upload and Load Data ===================
uploaded = files.upload()
zip_file = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('/content')
data_folder = '/content/BP_Data'
ppgn_df = pd.read_csv(os.path.join(data_folder, 'PPGn.csv'), header=None)
sbpn_df = pd.read_csv(os.path.join(data_folder, 'SBPn.csv'), header=None)
dbpn_df = pd.read_csv(os.path.join(data_folder, 'DBPn.csv'), header=None)


Saving BP_Data-20250409T091057Z-001.zip to BP_Data-20250409T091057Z-001 (1).zip


In [ ]:
# =================== STEP 4: Feature Extraction ===================
def extract_stft_features(signal, fs=1000):
    f, t, Zxx = stft(signal, fs=fs)
    return np.abs(Zxx).mean(axis=1)[:75]  # reduced dimension

fft_feats = np.abs(fft(ppgn_df.values, axis=1))[:, :150]
stft_feats = np.array([extract_stft_features(sig) for sig in ppgn_df.values])
interaction_feats = fft_feats[:, :75] * stft_feats[:, :75]
X_feat = np.hstack((fft_feats, stft_feats, interaction_feats))

target_df = pd.concat([sbpn_df, dbpn_df], axis=1)

# =================== STEP 5: Dimensionality Reduction ===================
pls = PLSRegression(n_components=12)
X_pls = pls.fit_transform(X_feat, target_df.values)[0]

In [ ]:
# =================== STEP 6: Normalization ===================
scaler_x = MinMaxScaler(feature_range=(-1, 1))
scaler_y = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler_x.fit_transform(X_pls)
y_scaled = scaler_y.fit_transform(target_df)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

In [ ]:
# =================== STEP 7: Quantum Layer Setup ===================
n_qubits = 4
dev1 = qml.device("default.qubit", wires=n_qubits)
dev2 = qml.device("default.qubit", wires=n_qubits)
dev3 = qml.device("default.qubit", wires=n_qubits)
dev4 = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev1, interface="tf")
def qnode1(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

@qml.qnode(dev2, interface="tf")
def qnode2(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

@qml.qnode(dev3, interface="tf")
def qnode3(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

@qml.qnode(dev4, interface="tf")
def qnode4(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

qlayer1 = KerasLayer(qnode1, {"weights": (12, n_qubits, 3)}, output_dim=n_qubits, trainable=True)
qlayer2 = KerasLayer(qnode2, {"weights": (10, n_qubits)}, output_dim=n_qubits, trainable=True)
qlayer3 = KerasLayer(qnode3, {"weights": (8, n_qubits, 3)}, output_dim=n_qubits, trainable=True)
qlayer4 = KerasLayer(qnode4, {"weights": (6, n_qubits)}, output_dim=n_qubits, trainable=True)


/usr/local/lib/python3.11/dist-packages/pennylane/qnn/keras.py:317: PennyLaneDeprecationWarning: The 'KerasLayer' class is deprecated and will be removed in v0.42. 
  warnings.warn(


In [ ]:
# =================== STEP 8: Hybrid Quantum-Classical Model ===================
inputs = keras.Input(shape=(12,))


x1 = qlayer1(inputs[:, :4])         # 0–3
x2 = qlayer2(inputs[:, 4:8])        # 4–7
x3 = qlayer3(inputs[:, 8:12])       # 8–11


# Attention using Lambda layers to wrap TensorFlow ops
x1_exp = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(x1)
x2_exp = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(x2)
att = layers.Attention()([x1_exp, x2_exp])
att = layers.Lambda(lambda x: tf.squeeze(x, axis=1))(att)

# Fourth quantum layer on attended output
q4_out = qlayer4(att)

# Concatenate all quantum outputs
x = layers.Concatenate()([x1, x2, x3, att, q4_out])

# Feedforward + LayerNorm
x = layers.Dense(64, activation="gelu")(x)
x = layers.LayerNormalization()(x)

# Residual Block
res = layers.Dense(128, activation="gelu")(x)
x = layers.LayerNormalization()(res)
x = layers.Dense(128, activation="gelu")(x)
x = layers.LayerNormalization()(x)
x = layers.Add()([x, res])

# Squeeze-and-Excitation Block (wrapped in Lambda layer)
se_input = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(x)
se = layers.GlobalAveragePooling1D()(se_input)
se = layers.Dense(32, activation='relu')(se)
se = layers.Dense(x.shape[-1], activation='sigmoid')(se)
x = layers.Multiply()([x, se])

# Final MLP layers
x = layers.Dropout(0.3)(x)
skip = layers.Dense(16, activation="gelu")(x)
x = layers.Concatenate()([x, skip])
outputs = layers.Dense(2)(x)

# Compile model
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay

lr_schedule = ExponentialDecay(
    initial_learning_rate=0.002,
    decay_steps=1000,
    decay_rate=0.9
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss=custom_sbp_focused_loss,
    metrics=["mae"]
)

# Remove reduce_lr from callbacks
history = model.fit(
    X_train, y_train,
    validation_split=0.15,
    epochs=100,
    batch_size=32,
    verbose=1,
    callbacks=[early_stop]
)


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.7503 - mae: 0.6460

/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 86s 758ms/step - loss: 0.7398 - mae: 0.6419 - val_loss: 0.1784 - val_mae: 0.3814
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - loss: 0.2728 - mae: 0.4298 - val_loss: 0.1494 - val_mae: 0.3166
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - loss: 0.1526 - mae: 0.3030 - val_loss: 0.1137 - val_mae: 0.2683
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - loss: 0.1149 - mae: 0.2632 - val_loss: 0.0892 - val_mae: 0.2334
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.1013 - mae: 0.2404 - val_loss: 0.0754 - val_mae: 0.2054
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 0.0846 - mae: 0.2178 - val_loss: 0.0681 - val_mae: 0.1896
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 0.0824 - mae: 0.2103 - val_loss: 0.0653 - val_mae: 0.1923
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - loss: 0.0778 - mae: 0.2056 - val_loss: 0.0627 - val_mae: 0.1889
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 126ms/step - loss: 0.069

In [ ]:
# =================== STEP 11: Evaluation ===================
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test)

sbp_mae = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
dbp_mae = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
sbp_rmse = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
dbp_rmse = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
sbp_r2 = r2_score(y_true[:, 0], y_pred[:, 0])
dbp_r2 = r2_score(y_true[:, 1], y_pred[:, 1])

print(f"SBP \u2794 MAE: {sbp_mae:.2f}, RMSE: {sbp_rmse:.2f}, R\u00b2: {sbp_r2:.2f}")
print(f"DBP \u2794 MAE: {dbp_mae:.2f}, RMSE: {dbp_rmse:.2f}, R\u00b2: {dbp_r2:.2f}")

/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


4/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step 

/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


7/7 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step
SBP ➔ MAE: 4.67, RMSE: 6.21, R²: 0.70
DBP ➔ MAE: 2.05, RMSE: 2.90, R²: 0.85
